In [444]:
import numpy as np
import matplotlib 
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn import tree
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [445]:
train = pd.read_csv("contest-train.csv", encoding="utf-8")
test = pd.read_csv("contest-test.csv", encoding="utf-8")
columns = ['Unnamed: 0', 'ID', 'Start Date', 'End Date', 'Start Mo/Yr', 'End Mo/Yr', 'Start FY', 'End FY', 'Individual Travel Hours', 'Rqst Tot Labor Hrs', 'Airfare', 'Hotel', 'Per Diem', 'Other', 'Estimated Individual Travel', 'Misc Expenses', 'Catering', 'Facility Rental', 'Direct Other Expenses', 'Describe Other Expenses', 'Direct Expense Impact', 'Rqst NPR Alloc', 'Rqst NPR OH', 'Cancel No Response', 'Created', 'Retroactive Start Date', 'Duplicates', 'Reporting Status']
#columns = ['Unnamed: 0', 'ID']
#columns=['Training Location','Organization','Home Office/Metro Area','Function Name','Is there a course with this certification?', 'Certification URL','Organization Number', 'Assigned To', 'Course or Event Name', 'Vendor Charge', 'Course Name', 'Course URL / Code', 'Unnamed: 0', 'ID', 'Request Status', 'Start Date', 'End Date', 'Start Mo/Yr', 'End Mo/Yr', 'Start FY', 'End FY', 'Individual Travel Hours', 'Rqst Tot Labor Hrs', 'Airfare', 'Hotel', 'Per Diem', 'Other', 'Estimated Individual Travel', 'Misc Expenses', 'Catering', 'Facility Rental', 'Direct Other Expenses', 'Describe Other Expenses', 'Direct Expense Impact', 'Rqst NPR Alloc', 'Rqst NPR OH', 'Cancel No Response', 'Created', 'Retroactive Start Date', 'Duplicates', 'Reporting Status']
#['ID', 'Course Name', 'Course URL / Code', 'Certification URL','Consolidated Course Name', 'Assigned To', 'Request Status', 'Start Date', 'End Date', 'Start Mo/Yr', 'End Mo/Yr', 'Start FY', 'End FY', 'Individual Travel Hours', 'Rqst Tot Labor Hrs', 'Airfare', 'Hotel', 'Per Diem', 'Other', 'Estimated Individual Travel', 'Misc Expenses', 'Catering', 'Facility Rental', 'Direct Other Expenses', 'Describe Other Expenses', 'Direct Expense Impact', 'Rqst NPR Alloc', 'Rqst NPR OH', 'Cancel No Response', 'Created', 'Retroactive Start Date', 'Duplicates', 'Reporting Status']
categorical = ['Training Source', 'Home Office/Metro Area', 'Organization Number', 'Organization', 'Capability', 'Function 2', 'Career Level', 'Function', 'Function Name', 'Title', 'Training Type', 'Training Provider', 'Training Delivery Type', 'Training Location', 'Course Name', 'Course URL / Code', 'Vendor Name', 'Conference Name', 'Course or Event Name', 'Certification Type', 'Certification Name', 'Certification URL', 'Is there a course with this certification?', 'Consolidated Course Name', 'Activity', 'Support Group', 'Assigned To', 'Request Status', 'Business Justification', 'What % of the conference is business development?', 'Travel Required']
train = train.drop(axis=1, inplace=False, labels = columns)
test = test.drop(axis=1, inplace=False, labels = columns)

full_set = pd.concat([train,test], axis=0)
full = pd.get_dummies(data = full_set, columns = categorical, drop_first = True)

#remove rows where Category is NaN
initial = full.dropna(subset=['Category'])
#output portion should be records where Category is NaN
x_out = full[full['Category'].isnull()]
x_out = x_out.drop(['Category'], axis = 1)

i = 0
cat = dict()
for name in initial['Category'].unique():
    cat[name] = i
    i += 1
    
inv_map = {v: k for k, v in cat.items()}
vfun = np.vectorize(inv_map.get)

## These only appear once for category
solo = initial.loc[(initial['Category'] == 'Pega') | (initial['Category'] == 'JDA') | (initial['Category'] == 'Recruiting')]
i_Names = initial[(initial['Category'] == 'Pega') | (initial['Category'] == 'JDA') | (initial['Category'] == 'Recruiting')].index
splitting = initial.drop(i_Names , inplace=False)
x_solo = solo.loc[:, solo.columns != 'Category']
y_solo = solo.loc[:, solo.columns == 'Category']
yip = splitting['Category'].copy()
nip = splitting.drop(['Category'], axis = 1)

# split up data by stratified random sampling to test results
x_train, x_test, y_train, y_test = train_test_split(nip, yip, test_size=.2, random_state=24, stratify=yip).copy()
x_train = x_train.append(x_solo)
y_train.rename('Category')
y_train = y_train.to_frame()
y_train = y_train.append(y_solo)
y_test.rename('Category')
y_test = y_test.to_frame()


#apply the map to Category
y_train_fin = pd.DataFrame(data = y_train['Category'].map(cat))
y_test_fin = pd.DataFrame(data = y_test['Category'].map(cat))

scoring = 'accuracy'

seed = 52
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train_fin.values.ravel())
prediction_trial = clf.predict(x_test)
print(accuracy_score(y_test_fin, prediction_trial))

0.9259259259259259


In [459]:
prediction = clf.predict(x_out)
prediction = vfun(prediction)
output = pd.read_csv("contest-test.csv", encoding="utf-8")
output = output.drop(output.columns[0], axis=1)
for i in range(0,len(prediction)):
    output.loc[i, 'Category'] = prediction[i]

In [461]:
output.to_csv('output.csv')

In [447]:

#.select_dtypes(exclude=['uint8'])
#param_can = {'max_depth':range(3,20)}
#kfold = model_selection.KFold(n_splits=10, random_state=seed)
#clf = GridSearchCV(tree.DecisionTreeClassifier(random_state=seed), param_can, n_jobs=4)
#clf.fit(x_train, y_train_fin.values.ravel())
#tree_model = clf.best_estimator_
#predicting = tree_model.predict(x_out)
#print(accuracy_score(dog, predicting))
#C = 4.3

#LR = LogisticRegression(C=C)
#LR.fit(x_train, y_train_fin.values.ravel())
#predictions = LR.predict(x_out)    
#print (accuracy_score(dog, predictions))

array([15, 15,  0, ..., 13, 11, 12])

In [247]:
# This block is only used for testing initial method

## These only appear once for category
#solo = train.loc[(train['Category'] == 'Pega') | (train['Category'] == 'JDA') | (train['Category'] == 'Recruiting')]
#i_Names = train[(train['Category'] == 'Pega') | (train['Category'] == 'JDA') | (train['Category'] == 'Recruiting')].index
#train.drop(i_Names , inplace=True)
#x_solo = solo.loc[:, solo.columns != 'Category']
#y_solo = solo.loc[:, solo.columns == 'Category']
#y = train.pop('Category')
#x = train
## split up data by stratified random sampling to test results
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.02, random_state=24, stratify=y).copy()
#x_train = x_train.append(x_solo)
#y_train.rename('Category')
#y_train = y_train.to_frame()
#y_train = y_train.append(y_solo)
#for col in x_test.columns:
#    x_test[col] = x_test[col].astype('category').cat.codes
#    x_test[col] = x_test[col].astype('category',copy=False)
        
#y_train = train.pop('Category')
#x_train = train
# map category values to a unique integer
#i = 0
#cat = dict()
#for name in y_train.tolist():
#    if name not in cat:
#        cat[name] = i
#        i += 1

#y_train_num = pd.DataFrame(data = y_train.map(cat))

#for col in x_train.columns:
#    if x_train[col].dtype == 'O':
#        x_train[col] = x_train[col].astype('category').cat.codes
#        x_train[col] = x_train[col].astype('category',copy=False)
        
#x_train['What % of the conference is business development?'] = x_train['What % of the conference is business development?'].astype('category').cat.codes
#x_train['What % of the conference is business development?'] = x_train['What % of the conference is business development?'].astype('category',copy=False)
#inv_map = {v: k for k, v in cat.items()}
#vfun = np.vectorize(inv_map.get)

#x_train.fillna(0, inplace=True)

#y_test = y_test.to_frame()
#y_test_num = pd.DataFrame(data = test['Category'].map(cat))

In [248]:
seed = 7
scoring = 'accuracy'
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
#models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC(gamma='auto')))
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=10, random_state=seed)
	cv_results = model_selection.cross_val_score(model, x_train, y_train_num.values.ravel(), cv=kfold, scoring=scoring)
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)

LR: 0.385076 (0.322085)
KNN: 0.378088 (0.336731)
CART: 0.456119 (0.340291)


In [260]:
result = pd.read_csv("results.csv", encoding="unicode_escape")
test_val = result.drop(axis=1, inplace=False, labels = columns)
for col in test_val.columns:
    if test_val[col].dtype == 'O':
        test_val[col] = test_val[col].astype('category').cat.codes
        test_val[col] = test_val[col].astype('category',copy=False)
test_val['What % of the conference is business development?'] = test_val['What % of the conference is business development?'].astype('category').cat.codes
test_val['What % of the conference is business development?'] = test_val['What % of the conference is business development?'].astype('category',copy=False)
results = result['Category']
test_val.pop('Category')
results = results.to_frame()
results_num = pd.DataFrame(data = results['Category'].map(cat))

clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train_num)
prediction = clf.predict(test_val)
prediction = vfun(prediction)
print(accuracy_score(results, prediction))
#print(confusion_matrix(results, prediction))

0.7113745831789552


In [250]:
param_can = {'max_depth':range(3,20)}
kfold = model_selection.KFold(n_splits=10, random_state=seed)
clf = GridSearchCV(tree.DecisionTreeClassifier(random_state=seed), param_can, n_jobs=4)
clf.fit(x_train, y_train_num)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


0.9119170984455959 {'max_depth': 19}


In [251]:
prediction = tree_model.predict(test_val)
prediction = vfun(prediction)
print(accuracy_score(results, prediction))

0.7098925527973323


In [252]:
#Knn not as good as decision tree

# finding best param for knn
#params = {'n_neighbors':[31, 32, 33, 34, 35, 36, 37, 38, 29, 30]}#5,6,7,8,9,10,11]} #36 best param

#knn = neighbors.KNeighborsRegressor()

#model = GridSearchCV(knn, params, cv=5)
#model.fit(x_train,y_train_num)
#model.best_params_

# predicting with knn
#neigh = KNeighborsClassifier(n_neighbors = 36)
#neigh.fit(x_train, y_train_num.values.ravel())
#test_predict = neigh.predict(test_val)
#print(accuracy_score(results_num, test_predict))


C = 4.3
LR = LogisticRegression(C=C)
LR.fit(x_train, y_train_num.values.ravel())
predictions = LR.predict(test_val) 
predictions = vfun(predictions)
print (accuracy_score(results, predictions))

0.7161911819192294
